Combine Sheps Center hospital files from 2016-2024 with standardized column names

In [77]:
import pandas as pd
import glob
import os
import re
from pathlib import Path

import warnings
warnings.filterwarnings("ignore", message="Cannot parse header or footer", module="openpyxl.worksheet.header_footer")

### Load excel files and concat into one big file

In [78]:
# Define the path to files
folder_path = '../data/yearly_hospital_lists/raw/'  # Update this if your path is different
excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

# Define name variants
acute_keywords = ['acute', 'acute hospitals', 'aucte hospitals'] # 2024 acute tab is misspelled
specialty_keywords = ['specialty']

# Storage for each category
acute_list = []
specialty_list = []

In [79]:
# Column mapping to standardize column names
column_map = {
    'id': 'ID',
    'name': 'NAME',
    'address': 'ADDRESS',
    'city': 'CITY',
    'state': 'STATE',
    'zip': 'ZIP',
    'zip code': 'ZIP',
    'fips': 'FIPS',
    'fips 2022': 'FIPS',
    'ruca': 'RUCA',
    'ruca code': 'RUCA',
    'forhp rural/urban 2018': 'RURAL_STATUS',
    'forhp rural/urban 2020': 'RURAL_STATUS',
    'forhp rural/urban 2021': 'RURAL_STATUS',
    'forhp/ rural 2022': 'RURAL_STATUS',
    'forhp rural 2024': 'RURAL_STATUS',
    'forhprural2021': 'RURAL_STATUS',
    'forhprural2022': 'RURAL_STATUS',
    'type': 'TYPE',
    's_type': 'TYPE',
    'pos total beds': 'TOTAL_BEDS',
    'total beds, pos': 'TOTAL_BEDS',
    'pos tot beds': 'TOTAL_BEDS',
    'cr total beds': 'TOTAL_BEDS',
    'acute beds, hcris': 'ACUTE_BEDS',
    'cr acute beds': 'ACUTE_BEDS',
    'cr beds': 'ACUTE_BEDS',
    'hcris acute beds': 'ACUTE_BEDS',
    'hcris beds': 'ACUTE_BEDS',
    # ignore these:
    'phone': None, 'phone number': None, 'beds, psf': None, 'psf beds': None,
}

core_columns = [
    'ID', 'NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP',
    'FIPS', 'RUCA', 'RURAL_STATUS', 'TOTAL_BEDS', 'ACUTE_BEDS', 'TYPE'
]

In [80]:
# Helper functions

def extract_year(filename):
    match = re.search(r'20\d{2}', filename)
    return match.group(0) if match else "Unknown"

def standardize_columns(df):
    new_cols = {
        col: column_map.get(col.strip().lower(), None)
        for col in df.columns
    }
    df = df.rename(columns=new_cols)
    return df[[col for col in core_columns if col in df.columns]]

def deduplicate_columns(df):
    seen = {}
    new_cols = []
    for col in df.columns:
        col_base = col
        count = seen.get(col_base, 0)
        new_col = f"{col_base}.{count}" if count else col_base
        new_cols.append(new_col)
        seen[col_base] = count + 1
    df.columns = new_cols
    return df

def load_and_clean(file_path, sheet_name, year, hospital_type):
    df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
    df = standardize_columns(df)
    df['YEAR'] = year
    df['HOSPITAL_TYPE'] = hospital_type
    return deduplicate_columns(df)

In [81]:
acute_frames = []
specialty_frames = []

for file_path in excel_files:
    print(os.path.basename(file_path))
    
    if os.path.basename(file_path).startswith('~$'):
        continue  # Skip temporary/lock files
        
    year = extract_year(Path(file_path).name)
    xls = pd.ExcelFile(file_path, engine='openpyxl')

    for sheet in xls.sheet_names:
        sheet_lower = sheet.strip().lower()

        if any(k in sheet_lower for k in acute_keywords):
            df = load_and_clean(file_path, sheet, year, 'Acute')
            acute_frames.append(df)

        elif any(k in sheet_lower for k in specialty_keywords):
            df = load_and_clean(file_path, sheet, year, 'Specialty')
            specialty_frames.append(df)

Hospital-List2023.xlsx
HospitalList2018aFinal.xlsx
RuralHospitalList2016.xlsx
HospitalList2022_revised4.27.2023.xlsx
HospitalList2019b.xlsx
HospitalList2020daFINAL.xlsx
HospitalList2017b.xlsx
r-Hospital-List2024.xlsx
HospitalList2021_revised4.27.2023.xlsx


In [82]:
acute_df = pd.concat(acute_frames, ignore_index=True)
specialty_df = pd.concat(specialty_frames, ignore_index=True)

In [83]:
# Combine the cleaned acute and specialty DataFrames
combined_df = pd.concat([acute_df, specialty_df], ignore_index=True)

In [84]:
# Convert ZIP to string, strip decimals, and pad with zeros if needed
combined_df['ZIP'] = (
    combined_df['ZIP']
    .astype(str)
    .str.replace(r'\.0$', '', regex=True)
    .str.zfill(5)
)

#### Check to make sure we didn't lose any hosptials in this process
The number of rows for each year and hospital type in combined_df should match those in our input excel files.

In [85]:
# Loop through files and print counts
for file_path in excel_files:
    filename = Path(file_path).name
    year = extract_year(filename)
    xls = pd.ExcelFile(file_path, engine='openpyxl')

    for sheet in xls.sheet_names:
        sheet_lower = sheet.strip().lower()

        if any(k in sheet_lower for k in acute_keywords):
            df = pd.read_excel(file_path, sheet_name=sheet, engine='openpyxl')
            print(f"{year} | {filename} | Acute     | {len(df)} rows")

        elif any(k in sheet_lower for k in specialty_keywords):
            df = pd.read_excel(file_path, sheet_name=sheet, engine='openpyxl')
            print(f"{year} | {filename} | Specialty | {len(df)} rows")

2023 | Hospital-List2023.xlsx | Acute     | 4518 rows
2023 | Hospital-List2023.xlsx | Specialty | 1421 rows
2018 | HospitalList2018aFinal.xlsx | Acute     | 4713 rows
2018 | HospitalList2018aFinal.xlsx | Specialty | 1380 rows
2016 | RuralHospitalList2016.xlsx | Acute     | 4768 rows
2016 | RuralHospitalList2016.xlsx | Specialty | 1372 rows
2022 | HospitalList2022_revised4.27.2023.xlsx | Acute     | 4591 rows
2022 | HospitalList2022_revised4.27.2023.xlsx | Specialty | 1406 rows
2019 | HospitalList2019b.xlsx | Acute     | 4663 rows
2019 | HospitalList2019b.xlsx | Specialty | 1381 rows
2020 | HospitalList2020daFINAL.xlsx | Acute     | 4613 rows
2020 | HospitalList2020daFINAL.xlsx | Specialty | 1379 rows
2017 | HospitalList2017b.xlsx | Acute     | 4741 rows
2017 | HospitalList2017b.xlsx | Specialty | 1390 rows
2024 | r-Hospital-List2024.xlsx | Acute     | 4497 rows
2024 | r-Hospital-List2024.xlsx | Specialty | 1457 rows
2021 | HospitalList2021_revised4.27.2023.xlsx | Acute     | 4591 rows


In [86]:
combined_df.pivot_table(index='YEAR', columns='HOSPITAL_TYPE', aggfunc='size', fill_value=0)

HOSPITAL_TYPE,Acute,Specialty
YEAR,,
2016,4768,1372
2017,4741,1390
2018,4713,1380
2019,4663,1381
2020,4613,1379
2021,4591,1381
2022,4591,1406
2023,4518,1421
2024,4497,1457


### Export csv

In [88]:
combined_df.to_csv('../data/yearly_hospital_lists/processed/combined_df.csv', index=False)